In [9]:
%pip install langchain langgraph langgraph-checkpoint-sqlite langchain_community requests termcolor wikipedia arxiv

Note: you may need to restart the kernel to use updated packages.


## Model

In [10]:
from services.model_service import ModelService

# Initialize the service with the model configuration
ollama_service = ModelService(model="llama3.1:8b-instruct-fp16")

## State Graph

In [11]:
from langgraph.graph.message import add_messages
from typing import Annotated, TypedDict, Any


class MultiAgentGraphState(TypedDict):
    input: str
    pm_response: Annotated[list, add_messages]
    virt_engineer_response: Annotated[list, add_messages]
    ocp_engineer_response: Annotated[list, add_messages]

## Tools

In [12]:
import json
from utils.general.tools import (
    get_tools_name,
    get_tools_description_json,
)

In [13]:
from tools.openshift.migration_plan import (
    create_migration_plan_tool,
    start_migration_plan_tool,
)

ocp_tools = [create_migration_plan_tool, start_migration_plan_tool]

ocp_tools_name = get_tools_name(ocp_tools)
ocp_tools_description_json = get_tools_description_json(ocp_tools)

ocp_tools_description = json.dumps(ocp_tools_description_json, indent=4)

print(ocp_tools_description)

[
    {
        "name": "create_migration_plan_tool",
        "description": "A tool to create a migration plan for moving virtual machines (VMs) using the forklift.konveyor.io API. This tool only requires VM names, and the corresponding IDs are fetched automatically.",
        "parameters": {
            "vm_names": {
                "param_type": "str",
                "description": "A list of VM names to migrate. This arg is mandatory.",
                "required": true
            },
            "name": {
                "param_type": "str",
                "description": "The name of the migration plan. This arg is mandatory.",
                "required": true
            },
            "source": {
                "param_type": "str",
                "description": "The source provider. Default is 'vmware'.",
                "required": false
            }
        }
    },
    {
        "name": "start_migration_plan_tool",
        "description": "A tool to start a migration for a

In [14]:
from tools.vsphere.vms import list_vms, retrieve_vm_details

virt_tools = [list_vms, retrieve_vm_details]

virt_tools_name = get_tools_name(virt_tools)
virt_tools_description_json = get_tools_description_json(virt_tools)

virt_tools_description = json.dumps(virt_tools_description_json, indent=4)

print(virt_tools_description)

[
    {
        "name": "list_vms",
        "description": "A wrapper around a vSphere utility for listing all available virtual machines (VMs). Useful for retrieving a list of VMs from the connected vSphere environment.",
        "parameters": {}
    },
    {
        "name": "retrieve_vm_details",
        "description": "A wrapper around a vSphere utility for extracting detailed information about a specific virtual machine (VM). Useful for retrieving the VM's operating system, resource allocations, and network configurations based on the provided VM name.",
        "parameters": {
            "vm_name": {
                "param_type": "str",
                "description": "The name of the virtual machine to retrieve details for.",
                "required": true
            }
        }
    }
]


## Agent Nodes

In [15]:
from agent.react_agent import ReactAgent


def virt_eng_node_function(state: MultiAgentGraphState, task: str):
    virt_agent = ReactAgent(
        state=state,
        role="virt_engineer",
        tools=virt_tools,
        ollama_service=ollama_service,
    )

    return virt_agent.react(user_request=task)


def ocp_eng_node_function(state: MultiAgentGraphState, task: str):
    ocp_agent = ReactAgent(
        state=state,
        role="ocp_engineer",
        tools=ocp_tools,
        ollama_service=ollama_service,
    )

    return ocp_agent.react(user_request=task)

## Workflow

In [16]:
from state.state_graph import create_graph, run_workflow
from memory.sqlite_saver import initialize_memory

vm_name = "database-tttcp"
task_virt = f"This is a SINGLE task: List the details for the vm named: {vm_name}! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!"
task_ocp = f"You should execute only one task. THIS IS A SINGLE TASK: Use the create_migration_plan_tool to create a migration plan for the vm named '{vm_name}'! REMEMBER: DON'T DO ANY ADDITIONAL STEP! DO NOT START OR EXECUTE THE MIGRATION YET! "

# Define a dictionary to map node names to their corresponding functions
agent_nodes = {
    "virt_engineer_node": lambda state: virt_eng_node_function(
        state=state, task=task_virt
    ),
    "ocp_engineer_node": lambda state: ocp_eng_node_function(
        state=state, task=task_ocp
    ),
}


memory = initialize_memory(":memory:")

# Step 2: Create the graph
graph = create_graph(
    MultiAgentGraphState,
    agent_nodes,
)

# Step 3: Define the input query and config
query = "Who's the USA's President?"
config = {"configurable": {"thread_id": "1"}}

# Step 4: Run the workflow
run_workflow(
    graph=graph,
    input_query=query,
    config=config,
    memory=memory,
    iterations=10,
    verbose=True,
)

<|start_header_id|>user<|end_header_id|>

This is a SINGLE task: List the details for the vm named: database-tttcp! Just do this and nothing else! DO NOT RETRIEVE THE DETAILS!<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

{
    "thought": "To get the list of virtual machines, I need to use the list_vms tool.",
    "action": "list_vms",
    "action_input": {}
}<|eot_id|>
<|python_tag|>list_vms.call({})
<|eom_id|>
Connected successfully to vCenter at vcsrs00-vc.infra.demo.redhat.com
VMs in the vSphere environment: roadshow-tpl-winweb01, winweb02-tttcp, winweb01-tttcp, database-tttcp, haproxy-tttcp, rhel93-tpl, roadshow-tpl-database, roadshow-tpl-winweb02, rhel86-tpl, rhel9-tpl
<|start_header_id|>ipython<|end_header_id|>

['roadshow-tpl-winweb01', 'winweb02-tttcp', 'winweb01-tttcp', 'database-tttcp', 'haproxy-tttcp', 'rhel93-tpl', 'roadshow-tpl-database', 'roadshow-tpl-winweb02', 'rhel86-tpl', 'rhel9-tpl']<|eot_id|>
<|start_header_id|>ipython<|end_header_id|>

{
    "observatio

/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


OpenShift Service connected. API is healthy.

1/8 Connected to Openshift...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



2/8 Now I have the source provider ID f19517d3-9207-4392-be1c-910b0724ebec...



...


5/8 Now I have the source datastore ID datastore-5031...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



7/8 Now I have the Storage and Network maps [{'id': 'vm-98459', 'name': 'database-tttcp'}]...



/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'forklift-inventory-openshift-mtv.apps.rosa-tttcp.1fe1.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/gsampaio/redhat/ai/virt-llm-agents/.conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ap


8/8 Migration plan RESPONSE - {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'Plan', 'metadata': {'annotations': {'populatorLabels': 'True'}, 'creationTimestamp': '2024-09-13T17:55:37Z', 'generation': 1, 'managedFields': [{'apiVersion': 'forklift.konveyor.io/v1beta1', 'fieldsType': 'FieldsV1', 'fieldsV1': {'f:spec': {'.': {}, 'f:map': {'.': {}, 'f:network': {}, 'f:storage': {}}, 'f:provider': {'.': {}, 'f:destination': {}, 'f:source': {}}, 'f:targetNamespace': {}, 'f:vms': {}}}, 'manager': 'OpenAPI-Generator', 'operation': 'Update', 'time': '2024-09-13T17:55:37Z'}], 'name': 'database-migration-plan', 'namespace': 'openshift-mtv', 'resourceVersion': '2895172', 'uid': '3fb08805-d661-47aa-aa6f-d6dd5903712d'}, 'spec': {'map': {'network': {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'NetworkMap', 'name': 'vmware-62tw4', 'namespace': 'openshift-mtv', 'uid': 'b8313c93-077f-40dc-8423-7237123ba44c'}, 'storage': {'apiVersion': 'forklift.konveyor.io/v1beta1', 'kind': 'StorageMa